## 1단계: 코드 불러오기 


In [ ]:
!pip install scikit-video
!git clone https://github.com/drminix/first-order-model
%cd /content/first-order-model

##2단계: 인공지능 신경망 파일 가져오기


영상제작에 필요한 사진, 동영상 및 인공지능 신경망 파일을 구글 드라이브로 공유 해 놓았습니다. 아래 폴더를 본인의 구글 드라이브에 연결해 주세요.

https://drive.google.com/drive/folders/1TIjufuJdPgkzvCQ3G1mICQZVCxWbCOFX?usp=sharing


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
target_folder = "/content/gdrive/My Drive/deepfake"

Mounted at /content/gdrive


## 3단계: 입력영상과 사진을 불러오기

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os
import skvideo.io
warnings.filterwarnings("ignore")

#입력사진
source_image = imageio.imread('/content/gdrive/My Drive/deepfake/obama.jpg')

#입력영상
driving_video = skvideo.io.vread('/content/gdrive/My Drive/deepfake/damedame.mp4')

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


def display(source, driving, generated=None):
     fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

     ims = []
     for i in range(len(driving)):
         cols = [source]
         cols.append(driving[i])
         if generated is not None:
             cols.append(generated[i])
         im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
         plt.axis('off')
         ims.append([im])

     ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
     plt.close()
     return ani

## 5단계: 영상 만들기

In [ ]:
from demo import load_checkpoints
import os
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path=os.path.join(target_folder,'vox-cpk.pth.tar'))

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(driving_video, predictions).to_html5_video())

100%|██████████| 836/836 [00:31<00:00, 26.57it/s]
